In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten,BatchNormalization

from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
seed = 16
filename = '../fer2013.csv'
label_map = ['Enfado', 'Asco', 'Miedo', 'Alegria', 'Tristeza', 'Sorpresa', 'Neutral']
names=['emocion','pixels','uso']
df=pd.read_csv(filename,names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emocion,pixels,uso
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [3]:
def getData(filname):
    # Las imagenes 48x48 en grayscale (de 0 a 255)
    # N = 35887 (28709 de entrenamiento y 2304 de testeo)
    Y_train = []
    X_train = []
    Y_test = []
    X_test = []
    
    first = True
    uso = 'Training\n'
    for line in open(filname):
        #This condition skips the first condition
        if first:
            first = False
        else:
            row = line.split(',')
            condition = str(row[2])
            if condition == uso:
                Y_train.append(int(row[0]))
                X_train.append([int(p) for p in row[1].split()])
            else:
                Y_test.append(int(row[0]))
                X_test.append([int(p) for p in row[1].split()])
                
    X_train, Y_train = np.array(X_train)/255.0 , np.array(Y_train)
    X_test, Y_test = np.array(X_test)/255.0 , np.array(Y_test)

    return X_train, X_test, Y_train, Y_test 

In [4]:
X_train, X_test, Y_train, Y_test= getData(filename)

In [5]:
cont_train = [0,0,0,0,0,0,0]
for j in Y_train:
    cont_train[j] += 1
cont_test = [0,0,0,0,0,0,0]
for j in Y_test:
    cont_test[j] += 1

print(cont_train)
print(cont_test)

[3995, 436, 4097, 7215, 4830, 3171, 4965]
[958, 111, 1024, 1774, 1247, 831, 1233]


In [6]:
N_train = X_train.shape[0]
N_test = X_test.shape[0]
X_train = X_train.reshape(N_train, 48, 48,1)
X_test = X_test.reshape(N_test, 48, 48,1)
# one hot encode output
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
class_num = Y_test.shape[1]

In [15]:
model = Sequential()


model.add(Conv2D(64,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


    

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(class_num))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_33 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
activation_33 (Activation)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
batch_normalization_34 (Batc (None, 24, 24, 128)      

In [18]:
epochs = 10
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
np.random.seed(16)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=32)

Train on 28709 samples, validate on 7178 samples
Epoch 1/10
28709/28709 [==============================] - 552s 19ms/step - loss: 1.5963 - accuracy: 0.3907 - val_loss: 1.4387 - val_accuracy: 0.4537
Epoch 2/10
28709/28709 [==============================] - 548s 19ms/step - loss: 1.3285 - accuracy: 0.5020 - val_loss: 1.4578 - val_accuracy: 0.4164
Epoch 3/10
28709/28709 [==============================] - 548s 19ms/step - loss: 1.2336 - accuracy: 0.5327 - val_loss: 1.3266 - val_accuracy: 0.4869
Epoch 4/10
28709/28709 [==============================] - 548s 19ms/step - loss: 1.1769 - accuracy: 0.5561 - val_loss: 1.2788 - val_accuracy: 0.5248
Epoch 5/10
 9824/28709 [=========>....................] - ETA: 5:20 - loss: 1.1228 - accuracy: 0.5778

KeyboardInterrupt: 